In [1]:
import pandas as pd
import json
import requests
import datetime as dt
from datetime import date, timedelta
import sys
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


In [2]:

start_date = date.today() - timedelta(days=3)
end_date = date.today() - timedelta(days=1)
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

print(start_date, end_date)

2024-08-24 2024-08-26


## 아바티 크롤링
아바티 로그인 및 데이터 수집
> iOS의 경우 1일 전 데이터가 안들어오는듯? 대시보드상에는 들어옴
`> 일단 API 요청했으니 구글시트로 temp

In [190]:
end_date = date.today().strftime('%Y-%m-%d')
end_date = date.today() - timedelta(days=8)
end_date = end_date.strftime('%Y-%m-%d')

In [188]:
print(end_date)

2024-08-21


In [56]:
start_date = "2024-05-23"
end_date = "2024-08-21"
platforms = {
    'ios':'d3a74845654442c38a77a1579c94e078',
    'aos':'6d5760ccfd9748d3a1ab2ffaa72c938f'
}
app_id = ''

In [57]:
#7일씩 가져옴. 마지막 날짜 기준
login_url = "https://partner.avatye.com/api/auth/check"
data_url = f"https://partner.avatye.com/api/cashButton/report/list?appID={app_id}&startDate={start_date}&endDate={end_date}&limit=999"
login_data = {
    'emailID' : "iehong@hecto.co.kr",
    'password' : "cash!1234"
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0',
    'Content-Type': 'application/json', 
}
def get_login(login_url,json,headers):
    response = session.post(login_url, json=login_data, headers=headers)
    if response.status_code == 200:
        return response
    else:
        False
        
# 웹 페이지 URL
def get_data(data_url):
    url = data_url
    # 웹 페이지의 HTML 가져오기
    response = requests.get(url)
    print(response)
    return response.json()

def convert_timezone(utc) :
    from datetime import datetime, timedelta

    utc_time = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%S.%fZ")
    kst_time = utc_time + timedelta(hours=9)
    return kst_time.strftime("%Y-%m-%d %H:%M:%S")

def get_avati_report(json_data):
    df = pd.json_normalize(
        json_data['list'],
        # record_path=['data'],
        # meta=['appID', 'appName'],
        # errors='ignore'
    )

    df['actionDateTime'] = df['actionDateTime'].apply(convert_timezone)

    return df

def set_header(data_url, app_id, start_date, end_date):
    data_url = f"https://partner.avatye.com/api/cashButton/report/list?appID={app_id}&startDate={start_date}&endDate={end_date}&limit=999"
    return data_url

with requests.Session() as session:
    response = get_login(login_url, json=login_data, headers=headers)
    if response == False:
        print("login_failed")
        sys.exit(1)
    else:
        print("login_success")
        pass
    
    key = 'ios'
    data_url = set_header(data_url,platforms[key],"2024-04-01","2024-08-26")
    json_data = get_data(data_url)

    result_data_ios = get_avati_report(json_data)
    result_data_ios['platform'] = key
    result_data_ios['actionDateTime'] = pd.to_datetime(result_data_ios['actionDateTime'])
    result_data_ios.rename(columns={'actionDateTime':'event_date'},inplace=True)

    key = 'aos'
    data_url = set_header(data_url,platforms[key],"2024-04-01","2024-08-26")
    json_data = get_data(data_url)

    result_data_aos = get_avati_report(json_data)
    result_data_aos['platform'] = key
    result_data_aos['actionDateTime'] = pd.to_datetime(result_data_aos['actionDateTime'])
    result_data_aos.rename(columns={'actionDateTime':'event_date'},inplace=True)
    

result_data = pd.concat([result_data_aos, result_data_ios],axis=0)
result_data.fillna(0)

result_data




login_success
<Response [200]>
<Response [200]>


C:\Users\Owner\AppData\Local\Temp\ipykernel_28056\2540511603.py:81: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_data.fillna(0)


,event_date,newAccount,dau,appDau,touchCount,touchPerUser,invitedCount,totalRevenue,totalExpense,expectedRevenue,totalSales,platform
0,2024-08-26,1268,13535,224360,1436670,106.0,None,527845,250000,527845,527845,aos
1,2024-08-25,915,15098,212267,1654764,109.0,None,621016,277000,621016,621016,aos
2,2024-08-24,655,14625,240652,1612670,110.0,None,662394,322000,662394,662394,aos
3,2024-08-23,657,15000,311720,1678359,111.0,None,600538,325000,600538,600538,aos
4,2024-08-22,670,15586,257423,1720375,110.0,None,567763,285000,567763,567763,aos
...,...,...,...,...,...,...,...,...,...,...,...,...
107,2024-05-11,0,0,0,0,NaN,None,0,0,0,0,ios
108,2024-05-10,0,0,0,0,NaN,None,0,0,0,0,ios
109,2024-05-09,0,0,0,0,NaN,None,0,0,0,0,ios
110,2024-05-08,0,0,0,0,NaN,None,0,0,0,0,ios


In [246]:
avati_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 0 to 106
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   actionDateTime   219 non-null    datetime64[ns]
 1   newAccount       219 non-null    int64         
 2   dau              219 non-null    int64         
 3   appDau           219 non-null    int64         
 4   touchCount       219 non-null    int64         
 5   touchPerUser     219 non-null    float64       
 6   invitedCount     219 non-null    int64         
 7   totalRevenue     219 non-null    int64         
 8   totalExpense     219 non-null    int64         
 9   expectedRevenue  219 non-null    int64         
 10  totalSales       219 non-null    int64         
 11  platform         219 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(9), object(1)
memory usage: 22.2+ KB


## 애드팝콘 revenue api

In [32]:
token = pd.read_csv('./access.csv')
ios = token.query('os=="ios"')['token'].values[0]
aos = token.query('os=="aos"')['token'].values[0]

token = {
    'aos':aos,
    'ios':ios
}

token['aos']

'81289be620467f6efd4e2d522febc1cd'

In [44]:
import datetime
import calendar

start_date = date.today() - timedelta(days=120)
end_date = date.today() - timedelta(days=1)
start_date = start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')


def get_daterange(year, month):
    
    start_date = datetime.date(year,month,1)
    end_date = datetime.date(year,month,calendar.monthrange(year,month)[1])
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')

    return start_date, end_date

def get_json_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # 요청이 성공하지 못하면 예외 발생
        data = response.json()  # 응답을 JSON 형식으로 파싱
        return data
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")  # HTTP 오류 처리
    except Exception as err:
        print(f"An error occurred: {err}")  # 기타 오류 처리
    return None

def get_table(token, start_date,end_date):
    url = f"https://report.ad-brix.com/v1/Revenue?access_token={token}&startDate={start_date}&endDate={end_date}&format=json"
    json_data = get_json_from_url(url)
    
    try:
        df = pd.json_normalize(
            json_data['Datas']
        )    

        df['ReportDate'] = pd.to_datetime(df['ReportDate'])
        df.rename(columns={'ReportDate':'event_date'},inplace=True)
        return df
    except Exception as e:
        print(json_data)
        print(e)

def init():
    token = pd.read_csv('./access.csv')
    ios = token.query('os=="ios"')['token'].values[0]
    aos = token.query('os=="aos"')['token'].values[0]
    token = {
        'aos':aos,
        'ios':ios
    }

    temp_23 = pd.DataFrame()
    temp_24 = pd.DataFrame()

    for m in range(11,13):
        start_date, end_date = get_daterange(2023,m)
        
        aos_df = get_table(token['aos'],start_date,end_date)
        ios_df = get_table(token['ios'],start_date,end_date)
        aos_df['platform'] = 'aos'
        ios_df['platform'] = 'ios'

        df = pd.concat([aos_df,ios_df],axis=0)
        temp_23 = pd.concat([temp_23,df],axis=0)

    for m in range(1,9):
        start_date, end_date = get_daterange(2024,m)
        
        aos_df = get_table(token['aos'],start_date,end_date)
        ios_df = get_table(token['ios'],start_date,end_date)
        aos_df['platform'] = 'aos'
        ios_df['platform'] = 'ios'

        df = pd.concat([aos_df,ios_df],axis=0)
        temp_24 = pd.concat([temp_24,df],axis=0)
    
    temp = pd.concat([temp_23,temp_24],axis=0)
    return temp

adpopcorn_df = init()

offerwall.Revenue : 오퍼월 총 매출 \
RCPM.Revenue : 간편적립 매출 \
ContentsClick.Revenue : 컨텐츠 클릭 매출

In [65]:
adpopcorn_df.query('event_date >= "2024-08-21"')

,event_date,TotalRevenue,DAU,Offerwall_OfferwallVisit,Offerwall_Conversion,Offerwall_CVR,Offerwall_Revenue,RCPM_Impression,RCPM_Click,RCPM_CTR,RCPM_eCPM,RCPM_Revenue,ContentsClick_Impression,ContentsClick_Click,ContentsClick_CVR,ContentsClick_Revenue,platform
20,2024-08-21,422344.0,0,7717,15603,202.19,282053.0,79312,43770,55.19,1768.85,140291.0,0,0,0.0,0.0,aos
21,2024-08-22,430045.0,0,8595,18541,215.72,278600.0,85087,46364,54.49,1779.88,151445.0,0,0,0.0,0.0,aos
22,2024-08-23,415158.0,0,7063,17264,244.43,268128.0,81935,45119,55.07,1794.47,147030.0,0,0,0.0,0.0,aos
23,2024-08-24,408495.0,0,6920,18183,262.76,265559.0,77884,44384,56.99,1835.24,142936.0,0,0,0.0,0.0,aos
24,2024-08-25,417921.0,0,8037,17909,222.83,272526.0,78960,45558,57.70,1841.38,145395.0,0,0,0.0,0.0,aos
25,2024-08-26,446790.0,0,9887,18579,187.91,283848.0,89697,47168,52.59,1816.58,162942.0,0,0,0.0,0.0,aos
26,2024-08-27,127739.0,0,0,8588,0.00,127739.0,0,0,0.00,0.00,0.0,0,0,0.0,0.0,aos
27,2024-08-28,0.0,0,0,0,0.00,0.0,0,0,0.00,0.00,0.0,0,0,0.0,0.0,aos
28,2024-08-29,0.0,0,0,0,0.00,0.0,0,0,0.00,0.00,0.0,0,0,0.0,0.0,aos
29,2024-08-30,0.0,0,0,0,0.00,0.0,0,0,0.00,0.00,0.0,0,0,0.0,0.0,aos


In [47]:
tt = adpopcorn_df.groupby(['event_date','platform'])['Offerwall.Revenue'].sum().reset_index().pivot(index='event_date',columns='platform',values='Offerwall.Revenue')
tt.query('index >= "2024-08-01" ')

platform,aos,ios
event_date,,
2024-08-01,326955.0,36937.0
2024-08-02,269167.0,29252.0
2024-08-03,277591.0,33424.0
2024-08-04,273317.0,36505.0
2024-08-05,319322.0,33113.0
2024-08-06,294325.0,43819.0
2024-08-07,261585.0,35666.0
2024-08-08,262352.0,25265.0
2024-08-09,246777.0,28609.0


https://report.ad-brix.com/v1/Revenue?access_token=81289be620467f6efd4e2d522febc1cd&startDate=2024-08-01&endDate=2024-08-02&format=json

## 핀크럭스

In [51]:
meta = {
    'pubkey':{
        'aos' : '911707',
        'ios' : '911708'
    },
    'account_id':'admin@hectobalso.com'
}

In [52]:
def set_meta(target_date,platform):
    url = 'https://ssl.pincrux.com/api/pub_report.pin'
    params = {
        'account_id': meta['account_id'],
        'api_token':'sffubhoaqzywecasaranovlybhgeunwc',
        'pubkey':meta['pubkey'][platform],
        'start_date':target_date,
        'end_date':target_date
    }

    return url, params

def get_data(url, params):
    response = requests.get(url,params=params)
    if response.status_code == 200:
        json_data = response.json()
    else:
        print('error')
        
    df = pd.json_normalize(
        json_data['item_list'],
    )

    return df

def get_run_loop(diff,start_date_dt,platform):
    temp_ = pd.DataFrame()
    for i in range(diff+1):
        target_date = start_date_dt + timedelta(days=i)
        target_date = datetime.datetime.strftime(target_date,'%Y-%m-%d')
        url, params = set_meta(target_date,platform)
        df = get_data(url, params)
        df['event_date'] = target_date
        df['platform'] = platform
        temp_ = pd.concat([temp_,df],axis=0)
    
    return temp_

def init():
    start_date = '2024-05-01'
    end_date = '2024-08-26'
    start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

    diff = end_date_dt - start_date_dt
    diff = diff.days

    aos_df = get_run_loop(diff,start_date_dt,platform='aos')
    ios_df = get_run_loop(diff,start_date_dt,platform='ios')
    
    fin_df = pd.concat([aos_df,ios_df],axis=0)
    fin_df['event_date'] = pd.to_datetime(fin_df['event_date'])
    
    return fin_df

pincrux_df = init()


In [235]:

pincrux_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22107 entries, 0 to 33
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   event_date    22107 non-null  datetime64[ns]
 1   platform      22107 non-null  object        
 2   pubkey        22107 non-null  float64       
 3   pub_nm        22107 non-null  object        
 4   appkey        22107 non-null  float64       
 5   app_nm        22107 non-null  object        
 6   click_cnt     22107 non-null  float64       
 7   complete_cnt  22107 non-null  float64       
 8   commission    22107 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(3)
memory usage: 1.7+ MB


## GCP 테이블 생성

In [58]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
from google.oauth2 import service_account
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)

In [59]:
def set_table(df, table_id):
    project_id = 'ballosodeuk'
    dataset_id = 'external_mart'
    table_id = table_id
    table_full_id = f'{project_id}.{dataset_id}.{table_id}'
    
    df.columns = df.columns.str.replace('.', '_')

    try:
        # BigQuery 클라이언트 생성
        client = bigquery.Client(project=project_id)
        
        # 테이블에 대한 job 설정
        job_config = bigquery.LoadJobConfig(
            write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # 기존 테이블을 대체
        )
        
        # DataFrame을 BigQuery 테이블로 업로드
        job = client.load_table_from_dataframe(df, table_full_id, job_config=job_config)
        job.result()  # 작업이 완료될 때까지 대기
        
        print("Table is set successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [60]:
set_table(adpopcorn_df,'offerwall_adpopcorn_optincome')

Table is set successfully.


In [61]:
set_table(result_data,'offerwall_avati_pnl')

An error occurred: Pandas requires version '10.0.1' or newer of 'pyarrow' (version '7.0.0' currently installed).


c:\Users\Owner\miniconda3\envs\pymc\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: invitedCount.
  warnings.warn(


In [62]:
set_table(pincrux_df,'offerwall_pincrux_revenue')

Table is set successfully.


In [58]:
result_data.fillna(0,inplace=True)

C:\Users\Owner\AppData\Local\Temp\ipykernel_43084\3727451725.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  result_data.fillna(0,inplace=True)
